# Typhoon Hagibis OLCI L1 and L2 case study plotter

    Version: 1.0
    Date:    14/10/2019
    Author:  Ben Loveday (Plymouth Marine Laboratory ) and Hayley Evers-King (EUMETSAT)
    Credit:  This code was developed for EUMETSAT under contracts for the Copernicus 
             programme.
    License: This code is offered as open source and free-to-use in the public domain, 
             with no warranty.

**What is this notebook for?**

This notebook shows plots OLCI L1 RGB and L2 complex water TSM products for Typhoon Hagibis.

**What specific tools does this notebook use?**

Beyond standard tools, the notebook imports tools for helping us to plot data (image_tools.py), and apply flags (validation_tools.py).

***

Lets begin....

Python is divided into a series of modules that each contain a series of methods for specific tasks. The box below imports all of the moduls we need to complete our plotting task

In [ ]:
%matplotlib inline

import os
import sys
import numpy as np
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib import gridspec
import matplotlib.ticker as mticker
import glob
from skimage import exposure
import warnings
warnings.filterwarnings('ignore')

# specific tools (which can be found here ../Hub_tools/)
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())) + '/Hub_Tools/')
import image_tools as img
import validation_tools as vt

## Function...

In [ ]:
def make_OLCI_plot(nc_dir, plot_extents=None, fsz=20,\
                              land_resolution='10m', vmin=-2, vmax=2,\
                              xsize=20, ysize=24, dpi=150,\
                              cmap=plt.cm.YlOrBr):
    '''
     Plots OLCI image from L1 (RGB) or L2 (TSM) source.
    '''
    # get our land mask from NaturalEarth
    land_poly = cfeature.NaturalEarthFeature('physical', 'land', land_resolution,
                                        edgecolor='k',
                                        facecolor=cfeature.COLORS['land'])
    # determine data type
    if '_WFR_' in nc_dir or '_WRR_' in nc_dir:
        ftype = 'L2'
    else:
        ftype = 'L1'
        
    # get the raster coords
    nc_file = os.path.join(nc_dir,'geo_coordinates.nc')
    nc_fid = xr.open_dataset(nc_file)
    LON = nc_fid.get('longitude').data
    LAT = nc_fid.get('latitude').data
    nc_fid.close()

    # get subset based on plot extents
    x1 = 0 ; x2 = -1; y1 = 0 ; y2 = -1
    if plot_extents:
        x1, x2, y1, y2 = img.subset_image(LAT, LON, plot_extents)
        LON = LON[x1:x2,y1:y2]
        LAT = LAT[x1:x2,y1:y2]
        #LON = img.reduce_image(LON, grid_factor=5)
        #LAT = img.reduce_image(LAT, grid_factor=5)
       
    # get the variables
    if ftype == 'L2':
        nc_file = os.path.join(nc_dir,'tsm_nn.nc')
        nc_fid = xr.open_dataset(nc_file)
        L2_VAR = np.squeeze(nc_fid.TSM_NN.data[x1:x2,y1:y2])
        nc_fid.close()
        #L2_VAR = img.reduce_image(L2_VAR, grid_factor=5)
        
        # gather flags
        flags_to_use = ['CLOUD', 'CLOUD_AMBIGUOUS', 'CLOUD_MARGIN',\
                 'INVALID', 'COSMETIC', 'SATURATED', 'SUSPECT',\
                 'HISOLZEN', 'HIGHGLINT', 'SNOW_ICE', 'AC_FAIL',\
                 'WHITECAPS']
        
        flag_fid = xr.open_dataset(os.path.join(nc_dir, 'wqsf.nc'))
        flags = flag_fid.get('WQSF').data[x1:x2,y1:y2]
        flag_names = flag_fid.get('WQSF').flag_meanings.split(' ')
        flag_vals = flag_fid.get('WQSF').flag_masks
        flag_fid.close()
        flag_mask = vt.flag_data_fast(flags_to_use, flag_names, flag_vals, flags)
        flag_mask = flag_mask.astype(float)
        flag_mask[flag_mask == 0.0] = np.nan
        #flag_mask = img.reduce_image(flag_mask, grid_factor=5)
        
        # apply the flags
        L2_VAR[np.isfinite(flag_mask)] = np.nan

    else:
        radiometry_type = 'Oa%s_radiance'
        num_channels = 11
        for rad_channel_number in range(1, num_channels+1):
            rad_channel = radiometry_type % (str(rad_channel_number).zfill(2))
            rad_file = os.path.join(nc_dir, rad_channel + '.nc') 
            rad_fid = xr.open_dataset(rad_file)
            exec("globals()['Ch%s'] = rad_fid.%s.data[x1:x2,y1:y2]" % (str(rad_channel_number).zfill(2),rad_channel))
            rad_fid.close()

        # make channels according to OLCI L1 tristimulus
        red = np.log10(1.0 + 0.01 * Ch01 + 0.09 * Ch02 + 0.35 * Ch03 + 0.04 \
                       * Ch04 + 0.01 * Ch05 + 0.59 * Ch06 + 0.85 * Ch07 + 0.12 \
                       * Ch08 + 0.07 * Ch09 + 0.04 * Ch10)
        
        green = np.log10(1.0 + 0.26 * Ch03 + 0.21 * Ch04 + 0.50 * Ch05 + Ch06 \
                         + 0.38 * Ch07 + 0.04 * Ch08 + 0.03 * Ch09 + 0.02 * Ch10)
        
        blue = np.log10(1.0 + 0.07 * Ch01 + 0.28 * Ch02 + 1.77 * Ch03 \
                        + 0.47 * Ch04 + 0.16 * Ch05)
        
        #red = img.reduce_image(red, grid_factor=5)
        #green = img.reduce_image(green, grid_factor=5)
        #blue = img.reduce_image(blue, grid_factor=5)
    
        # --------------------------- image manipulation -------------------------------

        # truncate image
        red = img.truncate_image(red) ; green = img.truncate_image(green) ; blue = img.truncate_image(blue)
    
        # build RGB array
        height = np.shape(red)[0] ; width = np.shape(red)[1] 
        image_array = np.zeros((height, width, 3), dtype=np.float32)
        image_array[..., 0] = red ; image_array[..., 1] = green ; image_array[..., 2] = blue

        # normalise array
        image_array = img.norm_image(image_array, contrast=[1, 1, 1], unhitch=True)
        
        # histogram channels
        image_array = exposure.equalize_adapthist(image_array, nbins=512)
        
        # --------------------------- image manipulation done ---------------------------
  
        # build color palette
        mesh_rgb = image_array[:, :-1, :]
        colorTuple = mesh_rgb.reshape((mesh_rgb.shape[0] * mesh_rgb.shape[1]), 3)
        colorTuple = np.insert(colorTuple, 3, 1.0, axis=1)
 
    # setup figure
    fig = plt.figure(figsize=(xsize, ysize), dpi=dpi)
    plt.rc('font', size=fsz)
 
    # setup axes
    gs = gridspec.GridSpec(3, 1, height_ratios=[20,0.5,0.5])
    gs.update(wspace=0.01, hspace=0.01)

    # setup plot 1
    axes_m = plt.subplot(gs[0,0], projection=ccrs.Mercator())
    axes_m.background_patch.set_facecolor('0.75')

    if ftype == 'L2':
        # plot the SST field
        p1 = axes_m.pcolormesh(LON, LAT, np.ma.masked_invalid(L2_VAR), cmap=cmap,\
                          vmin=vmin, vmax=vmax, zorder=10, transform=ccrs.PlateCarree())
    else:
        # plot the RGB array
        p1 = axes_m.pcolormesh(LON, LAT,\
                     red * np.nan, color=colorTuple ** 1.0,\
                     clip_on = True, edgecolors=None, zorder=0,\
                     transform=ccrs.PlateCarree())

    if plot_extents:
        axes_m.set_extent(plot_extents, ccrs.PlateCarree())
    
    # add some map embellishments
    if ftype == 'L2':
        axes_m.coastlines(resolution=land_resolution, color='black', linewidth=1)
        axes_m.add_feature(land_poly,color='0.25')
        
    g1 = axes_m.gridlines(draw_labels = True, color='0.25', linestyle='--', zorder=100)
    g1.xlabels_top = False ; g1.ylabels_right = False
    g1.xlabel_style = {'size': fsz, 'color': 'gray'}
    g1.ylabel_style = {'size': fsz, 'color': 'gray'}

    # setup plot2: colorbar
    if ftype == 'L2':
        axes_c = plt.subplot(gs[2,0])
        cbar = plt.colorbar(p1, cax=axes_c, orientation='horizontal', ticks=[-2, -1, 0, 1, 2])
        cbar.ax.tick_params(labelsize=fsz) 
        cbar.ax.set_xticklabels(['0.01', '0.1', '1', '10', '100'])
        cbar.set_label('Total Suspended Matter [g.m$^{-3}$] (OLCI NN)',fontsize=fsz)
    else:
        axes_c = plt.subplot(gs[2,0])
        axes_c.axis('off')
        
    return gs, axes_m, axes_c

# MAIN...

In [ ]:
if __name__ == '__main__':

    data_directory = '/home/jovyan/work/products/Hagibis'
    data_files = glob.glob(os.path.join(data_directory,'*.SEN3'))

    plot_extents = [135.50, 141.50, 32.0, 37.00]
    
    labels = ['OLCI L1 RGB Enhanced colour: 13/10/2019',\
              'OLCI L1 RGB Enhanced colour: 09/10/2019',\
              'OLCI L2 TSM Concentration: 09/10/2019',\
              'OLCI L1 RGB Enhanced colour: 12/10/2019',\
              'OLCI L2 TSM Concentration: 13/10/2019']
    
    fouts = ['Panel3','Panel1','Panel4','Panel2','Panel5']
    
    count = -1
    for data_file, label, fout in zip(data_files, labels, fouts):
        count = count + 1
        
        # make the plot: we will call this as a function as in contains a 'for' loop to make the plot
        fig, axis, colbar = make_OLCI_plot(data_file, plot_extents=plot_extents)
        plt.sca(axis)

        txt = plt.annotate(label, xy=(0.0, 1.005), xycoords='axes fraction',\
               size=24, color='k', zorder=100, annotation_clip=False)

        plt.savefig(fout)